In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sbn



In [ ]:
from matplotlib.pylab import rcParams
rcParams["figure.figsize"] = 10,6


 # Loading of Dataset

In [ ]:
bCancerDf = pd.read_csv("breast-cancer_csv.csv",na_values=["?"])
bCancerDf.head(10)



In [ ]:
##Age to Ordinal Data
##menopause to Categorical
##Tumor size ordinal
##inv-nodes to ordinal
##Node-caps to ordinal
##deg-malig to ordinal
##breast to categorical
##irradiat to categorical



In [ ]:
bCancerDf.shape


 # Checking For Null Values

In [ ]:
bCancerDf.isna().sum()


 # Decriptive Statistics of Data For Numeric Columns

In [ ]:
bCancerDf.describe(include=np.object)



In [ ]:
bCancerDf.describe(include=np.number)


 # Fix Some Columns With Values Not In Right Format

 Age,Tumor Nodes & Tumor Size were in ranges. So to help simplify things we computed the numerical averages for them.Even though this might affect the data we dont think it will be that have that much impact..

In [ ]:
def toInt(data,col_name):
    new_data = []
    for val in data:
        high,low = val.split("-")
        new_val = np.ceil((int(high) + int(low))/2)
        new_data.append(new_val)
    return pd.DataFrame(new_data,columns=[col_name])



In [ ]:
tobe_fixed = ["age","tumor-size","inv-nodes"]
new_dframes = []
for col in tobe_fixed:
    column = bCancerDf[col].values
    new_dframes.append(toInt(column,col))
final_bCancerDf = pd.concat(new_dframes,axis=1)



In [ ]:
final_bCancerDf.head()


 # Dropping Of Breast-Quad Column

 On Investigation, we found that breast-quadrants were incorrectly assigned to "RIGHT" even though the left breast was impacted.

In [ ]:
bCancerDf = bCancerDf.drop("breast-quad",axis=1)


 # Fixing Missing Values

In [ ]:
from sklearn.impute import SimpleImputer



In [ ]:
myImputer = SimpleImputer(strategy="most_frequent")



In [ ]:
bCancerDf.loc[:,["node-caps"]] = myImputer.fit_transform(bCancerDf.loc[:,["node-caps"]])



 # Converting Categorical Columns To Numeric Columns

In [ ]:
breast =  pd.get_dummies(data=bCancerDf.breast,prefix="breast",drop_first=True)
menopause = pd.get_dummies(data=bCancerDf.menopause,prefix="menopause",drop_first=True)
irradiat = pd.get_dummies(data=bCancerDf.irradiat,prefix="irradiat",drop_first=True)
node_caps = pd.get_dummies(data=bCancerDf["node-caps"],prefix="node-caps",drop_first=True)



In [ ]:
final_bCancerDf = pd.concat([final_bCancerDf,menopause,node_caps,breast,irradiat,bCancerDf["deg-malig"],bCancerDf["Class"]],1)



In [ ]:
from sklearn.preprocessing import LabelBinarizer



In [ ]:
labeler = LabelBinarizer()



In [ ]:
final_bCancerDf.iloc[:,[-1]] = labeler.fit_transform(final_bCancerDf.iloc[:,[-1]])



In [ ]:
final_bCancerDf.head()



 # Scaling of Features Excluding The Target to Deal With Euclidean Distance

In [ ]:
from sklearn.preprocessing import RobustScaler



In [ ]:
scaler = RobustScaler()



In [ ]:
final_bCancerDf.iloc[:,:-1] = scaler.fit_transform(final_bCancerDf.iloc[:,:-1])



In [ ]:
final_bCancerDf.head()


 # Checking For Correlation

In [ ]:
sbn.heatmap(final_bCancerDf.corr(),annot=True,linewidths=.5,robust=True)


# Preparing Of Data For Traing And Testing

In [ ]:
from sklearn.model_selection import train_test_split



In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(final_bCancerDf.iloc[:,:-1].values,final_bCancerDf.iloc[:,-1].values.ravel(),random_state=42)


# Supervised Ml

In [ ]:
from sklearn.metrics import confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Using The GridSearchCv To Find the Best Parameters For Our RandomForestClassifier

In [ ]:
forestClassifier = RandomForestClassifier()

In [ ]:
parameters= [{"n_estimators":[10,15,20],
    "criterion":['entropy'],
    "random_state":[32,42],
    "class_weight":["balanced"]},{"n_estimators":[10,15,20],
    "criterion":['entropy'],
    "random_state":[32,42],
    "class_weight":["balanced_subsample"]}]

In [ ]:
gridSearch = GridSearchCV(estimator=forestClassifier,param_grid=parameters,scoring="accuracy",cv=17,n_jobs=-1)

In [ ]:
gridSearch.fit(Xtrain,y=Ytrain)

In [ ]:
best_score = gridSearch.best_score_
bes_params = gridSearch.best_params_

In [ ]:
bes_params

In [ ]:
best_score

In [ ]:
grid_pred = gridSearch.predict(Xtest)

In [ ]:
grid_cnn = confusion_matrix(Ytest,grid_pred)
grid_cnn

# Using Naive Bayes

In [ ]:
naiveClassifier = GaussianNB()

In [ ]:
naiveClassifier.fit(Xtrain,Ytrain)

In [ ]:
naiveClassifier.score(Xtest,Ytest)


In [ ]:
naive_cnn = confusion_matrix(Ytest,naiveClassifier.predict(Xtest))
naive_cnn

 # Setup For Deep Learning Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.activations import relu,sigmoid
from keras.optimizers import Adam



In [ ]:
cancerRecallModel = Sequential()



In [ ]:
cancerRecallModel.add(Dense(150,activation=relu,input_dim=9))
cancerRecallModel.add(Dropout(0.5))
cancerRecallModel.add(BatchNormalization())
cancerRecallModel.add(Dense(100,activation=relu))
cancerRecallModel.add(Dropout(0.5))
cancerRecallModel.add(BatchNormalization())
cancerRecallModel.add(Dense(80,activation=relu))
cancerRecallModel.add(Dropout(0.5))
cancerRecallModel.add(BatchNormalization())
cancerRecallModel.add(Dense(80,activation=relu))
cancerRecallModel.add(Dropout(0.5))
cancerRecallModel.add(BatchNormalization())
cancerRecallModel.add(Dense(50,activation=relu))
cancerRecallModel.add(BatchNormalization())
cancerRecallModel.add(Dense(1,activation=sigmoid))
cancerRecallModel.summary()



In [ ]:
cancerRecallModel.compile(Adam(lr=0.001),loss="binary_crossentropy",metrics=["accuracy"])


 # Running Of Model

In [ ]:
cancerRecallModel.fit(x=Xtrain,y=Ytrain,epochs=100,validation_split=0.2)


 # Checks To See How Well Model Did

In [ ]:
cancerRecallModel.history.history.keys()



In [ ]:
validation_loss = np.mean(cancerRecallModel.history.history["val_loss"])
validation_acc = np.mean(cancerRecallModel.history.history["val_acc"])
train_loss = np.mean(cancerRecallModel.history.history["loss"])
train_acc = np.mean(cancerRecallModel.history.history["acc"])



In [ ]:
validation_acc



In [ ]:
validation_loss



In [ ]:
train_loss



In [ ]:
train_acc



In [ ]:
pred_classes = cancerRecallModel.predict_classes(Xtest)



In [ ]:
cnn =confusion_matrix(Ytest,pred_classes)



In [ ]:
cnn

